In [2]:
from akashiwo_data_scrape import *
from bs4 import BeautifulSoup
from tqdm import tqdm

In [6]:
# speacies id
spicies_ids = [45, 3]
spicies_names = ["Diaton.csv", "shtonela.csv"]

for id, names in zip(spicies_ids, spicies_names):
    pull_data(id, "2011/5/01", "2023/10/11", names)

In [96]:
colum_names = ['lat', 'lng', 'speciesValueAM', 'speciesValuePM', 
               'saisuiValueAM', 'saisuiValuePM', 'speciesNameKana', 
               'maxvalue', 'speciesId', 'icon_size']

for name in spicies_names:
    remove_colum(colum_names, name)

In [4]:
data = pd.read_csv("shtonela.csv")
pointIds = data["pointId"]
gatherYMDs = data["gatherYMD"]

In [19]:
main_data = {}
coordinate_data = {}
main_df = pd.DataFrame(main_data)
coordinate_df = pd.DataFrame(coordinate_data)
headers_to_skip_main = ["確定値／速報値", "事業・調査名"]
i = 0
for pointId, gatherYMD in zip(pointIds, gatherYMDs):
    html_data = requests.get(f"https://akashiwo.jp/private/akashiwoListInit.php?qpoint_id={str(pointId)}&qspecies_id=3&qgather_ymd_s=&qgather_ymd_e={str(gatherYMD)}")
    html_data.encoding = 'utf-8'
    soup = BeautifulSoup(html_data.text, 'html.parser')
    tables = soup.find_all('table')
    times_to_duplicate, main_df = parse_main_table(tables[1], headers_to_skip_main, main_df)
    coordinate_df = parse_coordinate_table(tables[0], coordinate_df, times_to_duplicate)
    if(i == 10): break
    i += 1

In [21]:
combined_data = coordinate_df.merge(main_df, left_index=True, right_index=True)
combined_data.to_csv("shatonela.csv", index=False)